In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # open cv library
import matplotlib.pyplot as plt #library

In [ ]:
# Importing necessary library
from keras.layers import Dense, Activation, Flatten,Dropout, Conv2D, Input
from keras.models import Sequential

In [ ]:
# import the necessary packages
from keras.preprocessing.image import img_to_array, ImageDataGenerator
from keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
trainingcsv_read=pd.read_csv('/kaggle/input/he-dl-trainimage/train.csv')

data= [] 
labels= [] 

import os
for dirname, _, filenames in os.walk('/kaggle/input/he-dl-trainimage/Train Images'):
    for filename in filenames:
        image = cv2.imread(os.path.join(dirname, filename))
        image = cv2.resize(image, (80,80),interpolation= cv2.INTER_CUBIC)
        #image = img_to_array(image)
        data.append(image)
        labels.append(trainingcsv_read['Class'][trainingcsv_read.loc[trainingcsv_read['Image']==filename].index[0]])

In [ ]:
# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

print("Labels before binarize :",labels[:10])

# Encode labels in column 'species'.
labels= label_encoder.fit_transform(labels)

print("Labels after binarise:", labels[:10])

In [ ]:
data = np.array(data) 
labels = np.array(labels)

from sklearn.model_selection import train_test_split
(X_train, X_test, Y_train,Y_test) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

Y_train = Y_train.T
Y_test = Y_test.T

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
#preprocessing the training image
X_train=X_train-np.mean(X_train)

In [ ]:
print(np.average(X_train))

In [ ]:
print(Y_train.classes)

In [ ]:
# model shared by navaneet
from keras.models import model
from keras.layers import Input,ZeroPadding2D,BatchNormalization,MaxPooling2D

X_input = Input((64,64,3))
X = ZeroPadding2D((3,3))(X_input)
X = Conv2D(32,(5,5),strides=(1,1),name='conv0')(X)
X = Activation('relu',name="a1")(X)
X = Conv2D(64,(5,5),strides=(1,1),name='conv1')(X)
X = BatchNormalization(axis=3,name='bn0')(X)
X = Activation('relu',name = 'a2')(X)
# X = Dropout(0.5)
X = MaxPooling2D((2,2),name='max_pooling')(X)

X = Flatten()(X)
X = Dense(1,activation='sigmoid',name='fc')(X)
model = model(inputs=X_input,outputs=X,name='model')

In [ ]:
# using the object rec model
from keras.models import Sequential
from keras.layers import Dropout,ZeroPadding2D,Activation,Conv2D,GlobalAveragePooling2D
# from tf.keras.layers import LeakyRelu

model = Sequential()

model.add(Conv2D(96,(3,3),padding='same',input_shape=(80,80,3),name = 'conv01'))
# model.add(LeakyRelu(0.01))
model.add(Conv2D(96,(3,3),padding='same',name = 'conv02'))
model.add(Activation('relu'))
model.add(Conv2D(96,(3,3),padding='same',strides=(2,2),name='conv03'))
model.add(Dropout(0.5))

model.add(Conv2D(192,(3,3),padding='same',name = 'conv04'))
model.add(Activation('relu'))
model.add(Conv2D(192,(3,3),padding='same',name = 'conv05'))
model.add(Activation('relu'))
model.add(Conv2D(192,(3,3),padding='same',strides=(2,2),name='conv06'))
model.add(Dropout(0.5))

model.add(Conv2D(192,(3,3),padding='same', name = 'conv07'))
model.add(Activation('relu'))
model.add(Conv2D(192,(1,1),padding='valid', name = 'conv08'))
model.add(Activation('relu'))
model.add(Conv2D(4,(1,1),padding='valid'))
model.add(Activation('relu'))

model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

#Accuracy is around 28%

In [ ]:
# using the object rec model with updated filter size
from keras.models import Sequential
from keras.layers import Dropout,ZeroPadding2D,Activation,Conv2D,GlobalAveragePooling2D


model = Sequential()

model.add(Conv2D(192,(3,3),padding='same',input_shape=(80,80,3),name = 'conv01'))
model.add(Activation('relu'))
model.add(Conv2D(192,(3,3),padding='same',name = 'conv02'))
model.add(Activation('relu'))
model.add(Conv2D(192,(3,3),padding='same',strides=(2,2),name='conv03'))
# model.add(Dropout(0.5))

model.add(Conv2D(256,(3,3),padding='same',name = 'conv04'))
model.add(Activation('relu'))
model.add(Conv2D(256,(3,3),padding='same',name = 'conv05'))
model.add(Activation('relu'))
model.add(Conv2D(256,(3,3),padding='same',strides=(2,2),name='conv06'))
model.add(Dropout(0.5))

model.add(Conv2D(512,(3,3),padding='same', name = 'conv07'))
model.add(Activation('relu'))
model.add(Conv2D(512,(1,1),padding='valid', name = 'conv08'))
model.add(Activation('relu'))
model.add(Conv2D(4,(1,1),padding='valid'))
model.add(Activation('relu'))


#add GlobalAveragePooling2D layer with softmax 
model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import SGD
learning_rate=0.01
weight_decay=0.0005
momentum=0.9
model.compile(loss='sparse_categorical_crossentropy',optimizer=SGD(lr=learning_rate,momentum=momentum,decay=weight_decay,nesterov=True),metrics=["sparse_categorical_accuracy"])

In [ ]:
from keras.callbacks import modelCheckpoint

# define additional training parameter
epochs = 100
batch_size = 256

#fit the model
filepath = './weight_improvements-{epoch:02d}-{sparse_categorical_accuracy:.2f}.h5'
checkpoint = modelCheckpoint(filepath,monitor="sparse_categorical_accuracy",verbose =1, mode='max',save_best_only=True)

#fit the model
model.fit(X_train,Y_train ,epochs=epochs,batch_size=batch_size,shuffle =True,callbacks=[checkpoint])

In [ ]:
learning_rate = 0.01
model.compile(loss = 'sparse_categorical_crossentropy',optimizer = Adam(lr = learning_rate),metrics=["sparse_categorical_accuracy"])

In [ ]:
import h5py

In [ ]:
from keras.model import vgg19

In [ ]:
from keras.callbacks import modelCheckpoint

In [ ]:
# define additional training parameter
epochs = 10
batch_size = 64

filepath = '../input/he-dl-trainimage/weight_improvements.{epoch:02d}-{accuracy:.2f}.h5'
checkpoint = modelCheckpoint(filepath,monitor="accuracy",verbose =1, mode='max')

#fit the model
model.fit(X_train,Y_train ,epochs=epochs,batch_size=batch_size,callbacks=[checkpoint])

In [ ]:
import keras
from keras.models import model
from keras.layers import Input,ZeroPadding2D,BatchNormalization,MaxPooling2D

# input layer is the same as our typical CNN model
inputs = Input(shape=(64,64,3))
num_classes= 4 
# # New Stuff Starts Here -------------------

tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(inputs)
tower_1 = Conv2D(64, (3,3), padding='same', activation= 'relu', name='t1_conv')(tower_1)

tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(inputs)
tower_2 = Conv2D(64, (5,5), padding='same', activation= 'relu', name='t2_conv')(tower_2)

tower_3 = MaxPooling2D( (1,3), padding='same',strides=(1,1))(inputs)
tower_3 = Conv2D(64, (1,1), padding='same', activation= 'relu', name='t3_conv')(tower_3)

x = keras.layers.concatenate([tower_1, tower_2, tower_3],axis =1)

#  Rest of the model, again, remains the same

x= Conv2D(8, (3,3)) (x)
x= Activation('relu')(x)
x= MaxPooling2D(pool_size=(2,2))(x)
x= Flatten()(x)

x= Dense(num_classes)(x)
output = Activation('softmax')(x)

model = model(inputs=X_input,outputs=x,name='model')

In [ ]:
#vgg network
from keras.models import Sequential
from keras.layers import Dropout,Activation,Conv2D,MaxPooling2D, Dense,GlobalAveragePooling2D
# import tf.keras.layers.LeakyRelu
# import tensorflow as tf
model= Sequential()

model.add(Conv2D(64, (3,3),padding='same',input_shape=(80,80,3),name = 'conv01'))
model.add(Activation('relu',name = 'actv01'))
# model.add(tf.keras.layers.LeakyRelu(0.01))
model.add(Conv2D(64, (3,3),padding='same',name = 'conv02'))
# model.add(LeakyRelu(0.01))
model.add(Activation('relu',name = 'actv02'))

model.add(MaxPooling2D((2,2),padding='same',strides=(2,2)))

model.add(Conv2D(128, (3,3),padding='same',name = 'conv03'))
model.add(Activation('relu',name = 'actv03'))
# model.add(LeakyRelu(0.01))
model.add(Conv2D(128, (3,3),padding='same',name = 'conv04'))
model.add(Activation('relu',name = 'actv04'))
# model.add(LeakyRelu(0.01))

model.add(MaxPooling2D((2,2),padding='same',strides=(2,2)))

model.add(Conv2D(128, (3,3),padding='same',name = 'conv05'))
model.add(Activation('relu',name = 'actv05'))
# model.add(LeakyRelu(0.01))
model.add(Conv2D(128, (3,3),padding='same',name = 'conv06'))
model.add(Activation('relu',name = 'actv06'))
# model.add(LeakyRelu(0.01))
model.add(Conv2D(128, (3,3),padding='same',name = 'conv07'))
model.add(Activation('relu',name = 'actv07'))
# model.add(LeakyRelu(0.01))
# model.add(Conv2D(128, (3,3),padding='same',name = 'conv08'))
# model.add(Activation('relu',name = 'actv08'))
# model.add(LeakyRelu(0.01))


model.add(MaxPooling2D((2,2),padding='same',strides=(2,2)))

model.add(Conv2D(512, (3,3),padding='same',name = 'conv09'))
model.add(Activation('relu',name = 'actv09'))
# model.add(LeakyRelu(0.01))
model.add(Conv2D(512, (3,3),padding='same',name = 'conv10'))
model.add(Activation('relu',name = 'actv10'))
# model.add(LeakyRelu(0.01))
model.add(Conv2D(512, (3,3),padding='same',name = 'conv11'))
model.add(Activation('relu',name = 'actv11'))
# model.add(LeakyRelu(0.01))
# model.add(Conv2D(512, (3,3),padding='same',name = 'conv12'))
# model.add(Activation('relu',name = 'actv12'))
# model.add(LeakyRelu(0.01))

model.add(MaxPooling2D((2,2),padding='same',strides=(2,2)))

model.add(Conv2D(128, (3,3),padding='same',name = 'conv13'))
model.add(Activation('relu',name = 'actv13'))
# model.add(LeakyRelu(0.01))
model.add(Conv2D(128, (3,3),padding='same',name = 'conv14'))
model.add(Activation('relu',name = 'actv14'))
# model.add(LeakyRelu(0.01))
model.add(Conv2D(128, (3,3),padding='same',name = 'conv15'))
model.add(Activation('relu',name = 'actv15'))
# model.add(LeakyRelu(0.01))
# model.add(Conv2D(128, (3,3),padding='same',name = 'conv16'))
# model.add(Activation('relu',name = 'actv16'))
# model.add(LeakyRelu(0.01))

model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu',name = 'actv17'))
model.add(Dropout(0.5))
model.add(Dense(1000,activation='relu',name = 'actv18'))
model.add(Dense(4,activation='relu',name = 'actv19'))


model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
# import tensorflow as tf
# run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)


learning_rate = 0.01
model.compile(loss = 'sparse_categorical_crossentropy',optimizer = Adam(lr = learning_rate),metrics=["sparse_categorical_accuracy"])#,options = run_opts)

In [ ]:
from keras.optimizers import SGD
learning_rate=0.01
weight_decay=0.0005
momentum=0.9
model.compile(loss='sparse_categorical_crossentropy',optimizer=SGD(lr=learning_rate,momentum=momentum,decay=weight_decay,nesterov=True),metrics=["sparse_categorical_accuracy"])

In [ ]:
from keras.callbacks import modelCheckpoint

# define additional training parameter
epochs = 100
batch_size = 256

#fit the model
filepath = './weight_improvements-{epoch:02d}-{sparse_categorical_accuracy:.2f}.h5'
checkpoint = modelCheckpoint(filepath,monitor="sparse_categorical_accuracy",verbose =1, mode='max',save_best_only=True)

#fit the model
model.fit(X_train,Y_train ,epochs=epochs,batch_size=batch_size,shuffle =True,callbacks=[checkpoint])

**Evaluate the model and predict on the test images set**

In [ ]:
def eval_Emodel(weights = None ):
    Emodel= Sequential()

    Emodel.add(Conv2D(64, (3,3),padding='same',input_shape=(80,80,3),name = 'conv01'))
    Emodel.add(Activation('relu',name = 'actv01'))

    Emodel.add(Conv2D(64, (3,3),padding='same',name = 'conv02'))
    Emodel.add(Activation('relu',name = 'actv02'))

    Emodel.add(MaxPooling2D((2,2),padding='same',strides=(2,2)))

    Emodel.add(Conv2D(128, (3,3),padding='same',name = 'conv03'))
    Emodel.add(Activation('relu',name = 'actv03'))

    Emodel.add(Conv2D(128, (3,3),padding='same',name = 'conv04'))
    Emodel.add(Activation('relu',name = 'actv04'))

    Emodel.add(MaxPooling2D((2,2),padding='same',strides=(2,2)))

    Emodel.add(Conv2D(128, (3,3),padding='same',name = 'conv05'))
    Emodel.add(Activation('relu',name = 'actv05'))

    Emodel.add(Conv2D(128, (3,3),padding='same',name = 'conv06'))
    Emodel.add(Activation('relu',name = 'actv06'))

    Emodel.add(Conv2D(128, (3,3),padding='same',name = 'conv07'))
    Emodel.add(Activation('relu',name = 'actv07'))

    Emodel.add(MaxPooling2D((2,2),padding='same',strides=(2,2)))

    Emodel.add(Conv2D(512, (3,3),padding='same',name = 'conv09'))
    Emodel.add(Activation('relu',name = 'actv09'))

    Emodel.add(Conv2D(512, (3,3),padding='same',name = 'conv10'))
    Emodel.add(Activation('relu',name = 'actv10'))

    Emodel.add(Conv2D(512, (3,3),padding='same',name = 'conv11'))
    Emodel.add(Activation('relu',name = 'actv11'))

    Emodel.add(MaxPooling2D((2,2),padding='same',strides=(2,2)))

    Emodel.add(Conv2D(128, (3,3),padding='same',name = 'conv13'))
    Emodel.add(Activation('relu',name = 'actv13'))

    Emodel.add(Conv2D(128, (3,3),padding='same',name = 'conv14'))
    Emodel.add(Activation('relu',name = 'actv14'))

    Emodel.add(Conv2D(128, (3,3),padding='same',name = 'conv15'))
    Emodel.add(Activation('relu',name = 'actv15'))

    Emodel.add(Dropout(0.5))
    Emodel.add(Dense(4096,activation='relu',name = 'actv17'))
    Emodel.add(Dropout(0.5))
    Emodel.add(Dense(1000,activation='relu',name = 'actv18'))
    Emodel.add(Dense(4,activation='relu',name = 'actv19'))


    Emodel.add(GlobalAveragePooling2D())
    Emodel.add(Activation('softmax'))
    
    # load the weights
    if weights:
        Emodel.load_weights(weights)
    
    # return model
    return model

In [ ]:
from keras.optimizers import SGD

#define hyperparameter
learning_rate=0.01
weight_decay=0.0005
momentum=0.9

# define weights and build model
weights = './weight_improvements-100-0.59.h5'
Emodel = allcnn(weights)

Emodel.compile(loss='sparse_categorical_crossentropy',optimizer=SGD(lr=learning_rate,momentum=momentum,decay=weight_decay,nesterov=True),metrics=["sparse_categorical_accuracy"])